# Demographic Summary

In [373]:
#| echo: false
import sys
import pandas as pd
sys.path.append('..')
import globalpaths
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe_connected'

In [374]:
#| echo: false
control_se = globalpaths.df_ControlTotal_SE_AllCounties
model_se = globalpaths.df_se_file

In [375]:
#| echo: false
year = int(globalpaths.DemographicYear)
control_cos = ['Box Elder', 'Weber - WFRC', 'Davis', 'Salt Lake', 'Utah']
control_cols = ['CO_NAME','YEAR', 'HH','HH_Pop','HH_Size','Job_HH','RETL','FOOD','MANU','WSLE','OFFI','GVED','HLTH','OTHR','AGRI','MING','CONS','HBJ']
model_cos = ['BOX ELDER', 'WEBER', 'DAVIS', 'SALT LAKE', 'UTAH']
model_cols = ['CO_NAME', 'TOTHH','HHPOP', 'ALLEMP', 'RETL','FOOD','MANU','WSLE','OFFI','GVED','HLTH','OTHR','AGRI','MING','CONS','HBJ', 'ENROL_ELEM', 'ENROL_MIDL','ENROL_HIGH']

In [376]:
#| echo: false
control_filtered = control_se.loc[(control_se['YEAR'] == year) & (control_se['CO_NAME'].isin(control_cos)), control_cols]
control_filtered = control_filtered.rename(columns={'HH':'TOTHH', 'HH_Pop':'HHPOP'}).drop(columns={'YEAR'})
condition = control_filtered['CO_NAME'].isin(control_cos)
control_filtered.loc[condition, 'CO_NAME'] = pd.Series(['BOX ELDER', 'DAVIS', 'SALT LAKE', 'UTAH', 'WEBER'], index = control_filtered[condition].index)
control_filtered['ALLEMP'] = control_filtered.iloc[:, control_filtered.columns.get_loc('RETL'): control_filtered.columns.get_loc('HBJ') + 1].sum(axis=1)
control_filtered['Type'] = 'ControlTotal'

In [377]:
#| echo: false
model_filtered = model_se.loc[model_se['CO_NAME'].isin(model_cos), model_cols]
model_filtered = model_filtered.groupby('CO_NAME').sum().reset_index()
model_filtered['HH_Size'] = model_filtered['HHPOP'] / model_filtered['TOTHH']
model_filtered['Job_HH'] = model_filtered['ALLEMP'] / model_filtered['TOTHH']
model_filtered['Type'] = 'Model'

In [378]:
#| echo: false
control_model = pd.concat([control_filtered, model_filtered], axis=0)

In [379]:
#| echo: false
control_model_pivot = control_model_long.pivot_table(index=['CO_NAME', 'Variable'], columns='Type', values='Value').reset_index()
control_model_pivot.columns.name = None
control_model_pivot['Diff'] = control_model_pivot['Model'] - control_model_pivot['ControlTotal']
control_model_pivot['%Diff'] = (control_model_pivot['Model'] - control_model_pivot['ControlTotal']) / control_model_pivot['ControlTotal'] * 100

In [380]:
#| echo: false
var_order = ['TOTHH','HHPOP', 'HH_Size','ALLEMP', 'RETL','FOOD','MANU','WSLE','OFFI','GVED','HLTH','OTHR','AGRI','MING','CONS','HBJ', 'Job_HH','ENROL_ELEM', 'ENROL_MIDL','ENROL_HIGH']

In [381]:
#| echo: false
def get_co_demo(co_name, pivot, variable_order):
    demo = pivot[pivot['CO_NAME'] == co_name]
    demo['Variable'] = pd.Categorical(demo['Variable'], categories=variable_order, ordered=True)
    demo = demo.sort_values(by='Variable')
    demo = demo.reset_index(drop=True).drop(columns='CO_NAME')
    demo['ControlTotal'] = demo['ControlTotal'].astype(float)
    demo['Diff'] = demo['Diff'].astype(float)
    demo['%Diff'] = demo['%Diff'].astype(float)
    demo = demo.round(0)
    return demo

In [382]:
def plot_job_sectors(co_name, long):
    sectors = ['RETL','FOOD','MANU','WSLE','OFFI','GVED','HLTH','OTHR','AGRI','MING','CONS','HBJ']
    bar_sectors = long.loc[long['CO_NAME'] == co_name]
    bar_sectors = bar_sectors.loc[bar_sectors['Variable'].isin(sectors)]

    df = px.data.tips()
    fig = px.histogram(bar_sectors, x="Variable", y="Value", text_auto='.2s',
                 color='Type', barmode='group',
                 height=400)
    fig.update_layout(
        xaxis_title="Job Type",
        yaxis_title="Number of Jobs",
        legend_title="Data Type"
    )
    #fig.update_xaxes(tickangle=90)
    fig.show()

## Box Elder

```{=html}
<details>
<summary>Expand Table</summary>
```

In [383]:
#| echo: False
#| warning: False
box_elder_demo = get_co_demo('BOX ELDER', control_model_pivot, var_order)
markdown_df = box_elder_demo.to_markdown(index=False)
display(Markdown(markdown_df))

C:\Users\cday\AppData\Local\Temp\ipykernel_21028\2056949888.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



| Variable   |   ControlTotal |   Model |   Diff |   %Diff |
|:-----------|---------------:|--------:|-------:|--------:|
| TOTHH      |          18369 |   10130 |  -8238 |     -45 |
| HHPOP      |          56542 |   30296 | -26246 |     -46 |
| HH_Size    |              3 |       3 |     -0 |      -3 |
| ALLEMP     |          30480 |   16973 | -13507 |     -44 |
| RETL       |           2756 |    1754 |  -1002 |     -36 |
| FOOD       |           1663 |    1111 |   -552 |     -33 |
| MANU       |           6460 |    2522 |  -3938 |     -61 |
| WSLE       |           2724 |     715 |  -2009 |     -74 |
| OFFI       |            993 |     712 |   -280 |     -28 |
| GVED       |           3399 |    2312 |  -1087 |     -32 |
| HLTH       |           2022 |    1434 |   -588 |     -29 |
| OTHR       |           5655 |    4234 |  -1420 |     -25 |
| AGRI       |           1539 |     390 |  -1149 |     -75 |
| MING       |             67 |      17 |    -50 |     -75 |
| CONS       |           2261 |    1254 |  -1007 |     -45 |
| HBJ        |            941 |     518 |   -423 |     -45 |
| Job_HH     |              2 |       2 |      0 |       1 |
| ENROL_ELEM |            nan |    2703 |    nan |     nan |
| ENROL_MIDL |            nan |    1093 |    nan |     nan |
| ENROL_HIGH |            nan |    1661 |    nan |     nan |

```{=html}
</details>
```

In [384]:
#| echo: False
#| label: fig-be-sector
#| fig-cap: "Box Elder Control Total vs. Modeled Number of Jobs by Job Type"
plot_job_sectors('BOX ELDER', control_model_long)

## Weber

```{=html}
<details>
<summary>Expand Table</summary>
```

In [385]:
#| warning: false
#| echo: false
weber_demo = get_co_demo('WEBER', control_model_pivot, var_order)
markdown_df = weber_demo.to_markdown(index=False)
display(Markdown(markdown_df))

C:\Users\cday\AppData\Local\Temp\ipykernel_21028\2056949888.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



| Variable   |   ControlTotal |   Model |   Diff |   %Diff |
|:-----------|---------------:|--------:|-------:|--------:|
| TOTHH      |          86141 |   86141 |      0 |       0 |
| HHPOP      |         249841 |  249842 |      1 |       0 |
| HH_Size    |              3 |       3 |      0 |       0 |
| ALLEMP     |         142482 |  142482 |      0 |       0 |
| RETL       |          15279 |   15279 |      0 |       0 |
| FOOD       |           8250 |    8250 |      0 |       0 |
| MANU       |          15224 |   15224 |      0 |       0 |
| WSLE       |           8189 |    8189 |      0 |       0 |
| OFFI       |           9083 |    9083 |      0 |       0 |
| GVED       |          23865 |   23865 |      0 |       0 |
| HLTH       |          14497 |   14497 |      0 |       0 |
| OTHR       |          34562 |   34562 |      0 |       0 |
| AGRI       |           1482 |    1481 |     -1 |      -0 |
| MING       |            143 |     143 |      0 |       0 |
| CONS       |           8672 |    8672 |      0 |       0 |
| HBJ        |           3236 |    3233 |     -3 |      -0 |
| Job_HH     |              2 |       2 |      0 |       0 |
| ENROL_ELEM |            nan |   26958 |    nan |     nan |
| ENROL_MIDL |            nan |   11524 |    nan |     nan |
| ENROL_HIGH |            nan |   11230 |    nan |     nan |

```{=html}
</details>
```

In [386]:
#| echo: False
#| label: fig-we-sector
#| fig-cap: "Weber Control Total vs. Modeled Number of Jobs by Job Type"
plot_job_sectors('WEBER', control_model_long)

## Davis

```{=html}
<details>
<summary>Expand Table</summary>
```

In [387]:
#| warning: false
#| echo: false
dave_demo = get_co_demo('DAVIS', control_model_pivot, var_order)
markdown_df = dave_demo.to_markdown(index=False)
display(Markdown(markdown_df))

C:\Users\cday\AppData\Local\Temp\ipykernel_21028\2056949888.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



| Variable   |   ControlTotal |   Model |   Diff |   %Diff |
|:-----------|---------------:|--------:|-------:|--------:|
| TOTHH      |         110496 |  110496 |     -0 |      -0 |
| HHPOP      |         356443 |  356442 |     -0 |      -0 |
| HH_Size    |              3 |       3 |     -0 |      -0 |
| ALLEMP     |         197873 |  197853 |    -20 |      -0 |
| RETL       |          20064 |   20064 |      0 |       0 |
| FOOD       |          10863 |   10863 |      0 |       0 |
| MANU       |          14769 |   14769 |      0 |       0 |
| WSLE       |          10730 |   10730 |      0 |       0 |
| OFFI       |          18991 |   18991 |      0 |       0 |
| GVED       |          40714 |   40719 |      5 |       0 |
| HLTH       |          15859 |   15859 |      0 |       0 |
| OTHR       |          46724 |   46724 |      0 |       0 |
| AGRI       |            957 |     956 |     -1 |      -0 |
| MING       |            150 |     150 |      0 |       0 |
| CONS       |          13618 |   13618 |     -0 |      -0 |
| HBJ        |           4434 |    4431 |     -3 |      -0 |
| Job_HH     |              2 |       2 |      0 |       0 |
| ENROL_ELEM |            nan |   41660 |    nan |     nan |
| ENROL_MIDL |            nan |   18712 |    nan |     nan |
| ENROL_HIGH |            nan |   17226 |    nan |     nan |

```{=html}
</details>
```

In [388]:
#| echo: False
#| label: fig-da-sector
#| fig-cap: "Davis Control Total vs. Modeled Number of Jobs by Job Type"
plot_job_sectors('DAVIS', control_model_long)

## Salt Lake

```{=html}
<details>
<summary>Expand Table</summary>
```

In [389]:
#| warning: false
#| echo: false
sl_demo = get_co_demo('SALT LAKE', control_model_pivot, var_order)
markdown_df = sl_demo.to_markdown(index=False)
display(Markdown(markdown_df))

C:\Users\cday\AppData\Local\Temp\ipykernel_21028\2056949888.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



| Variable   |     ControlTotal |            Model |   Diff |   %Diff |
|:-----------|-----------------:|-----------------:|-------:|--------:|
| TOTHH      | 399636           | 399634           |     -2 |      -0 |
| HHPOP      |      1.15899e+06 |      1.15898e+06 |     -8 |      -0 |
| HH_Size    |      3           |      3           |      0 |       0 |
| ALLEMP     | 950865           | 949309           |  -1556 |      -0 |
| RETL       |  85849           |  85852           |      3 |       0 |
| FOOD       |  56206           |  56208           |      2 |       0 |
| MANU       |  59851           |  59852           |      1 |       0 |
| WSLE       |  84870           |  84870           |      0 |       0 |
| OFFI       | 118808           | 118810           |      2 |       0 |
| GVED       | 140842           | 140844           |      2 |       0 |
| HLTH       |  73790           |  73791           |      1 |       0 |
| OTHR       | 248020           | 248021           |      1 |       0 |
| AGRI       |   1211           |   1209           |     -2 |      -0 |
| MING       |   3526           |   3526           |     -0 |      -0 |
| CONS       |  52061           |  52058           |     -3 |      -0 |
| HBJ        |  25831           |  25823           |     -8 |      -0 |
| Job_HH     |      2           |      2           |     -0 |      -0 |
| ENROL_ELEM |    nan           | 106386           |    nan |     nan |
| ENROL_MIDL |    nan           |  44618           |    nan |     nan |
| ENROL_HIGH |    nan           |  55514           |    nan |     nan |

```{=html}
</details>
```

In [390]:
#| echo: False
#| label: fig-sl-sector
#| fig-cap: "Salt Lake Control Total vs. Modeled Number of Jobs by Job Type"
plot_job_sectors('SALT LAKE', control_model_long)

## Utah

```{=html}
<details>
<summary>Expand Table</summary>
```

In [391]:
#| warning: false
#| echo: false
utah_demo = get_co_demo('UTAH', control_model_pivot, var_order)
markdown_df = utah_demo.to_markdown(index=False)
display(Markdown(markdown_df))

C:\Users\cday\AppData\Local\Temp\ipykernel_21028\2056949888.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



| Variable   |   ControlTotal |   Model |   Diff |   %Diff |
|:-----------|---------------:|--------:|-------:|--------:|
| TOTHH      |         178928 |  178923 |     -5 |      -0 |
| HHPOP      |         628931 |  628926 |     -5 |      -0 |
| HH_Size    |              4 |       4 |      0 |       0 |
| ALLEMP     |         377067 |  376875 |   -192 |      -0 |
| RETL       |          43048 |   42976 |    -72 |      -0 |
| FOOD       |          20840 |   20837 |     -3 |      -0 |
| MANU       |          21610 |   21610 |      0 |       0 |
| WSLE       |          15416 |   15352 |    -64 |      -0 |
| OFFI       |          54354 |   54355 |      1 |       0 |
| GVED       |          58393 |   58387 |     -6 |      -0 |
| HLTH       |          32868 |   32868 |      0 |       0 |
| OTHR       |          85420 |   85402 |    -18 |      -0 |
| AGRI       |           3462 |    3440 |    -22 |      -1 |
| MING       |            667 |     667 |      0 |       0 |
| CONS       |          30203 |   30206 |      3 |       0 |
| HBJ        |          10786 |   10778 |     -8 |      -0 |
| Job_HH     |              2 |       2 |     -0 |      -0 |
| ENROL_ELEM |            nan |   79682 |    nan |     nan |
| ENROL_MIDL |            nan |   32449 |    nan |     nan |
| ENROL_HIGH |            nan |   34727 |    nan |     nan |

```{=html}
</details>
```

In [394]:
#| echo: False
#| label: fig-ut-sector
#| fig-cap: "Utah Control Total vs. Modeled Number of Jobs by Job Type"
plot_job_sectors('UTAH', control_model_long)

## Checking SE File for inconsistencies:

Below are the TAZIDs where the Sum of Employment Categories are not within 0.5 of the ALLEMP value:

In [393]:
#| echo: False
model_cols2 = model_cols
model_cols2.append('Z')
model_filtered2 = model_se.loc[model_se['CO_NAME'].isin(model_cos), model_cols2]
model_filtered2['ALLEMP2'] = model_filtered2.iloc[:, model_filtered2.columns.get_loc('RETL'): model_filtered2.columns.get_loc('HBJ') + 1].sum(axis=1)
#z_values = model_filtered2.loc[model_filtered2['ALLEMP'] != model_filtered2['ALLEMP2'], 'Z'].tolist()
z_values = model_filtered2.loc[abs(model_filtered2['ALLEMP'] - model_filtered2['ALLEMP2']) > 0.5, 'Z'].tolist()
display(z_values)

[582, 583, 598, 677, 683, 684, 689, 740, 775, 794, 924]